In [1]:
import cvxpy as cp
import numpy as np
import time
import clarabel
import torch
import osqp
import osqp_cuda
from zap.admm import ADMMSolver
from zap.conic.cone_bridge import ConeBridge
import scipy.sparse as sp
from experiments.conic_solve.benchmarks.max_flow_benchmark import MaxFlowBenchmarkSet

from zap.conic.cone_utils import get_standard_conic_problem
from osqp import default_algebra



In [2]:
default_algebra()

'cuda'

In [2]:
## Create a large problem that is valid using max flow benchmark set 
n = 10000
base_seed = 42
max_flow_benchmark = MaxFlowBenchmarkSet(num_problems=1, n=n, base_seed=base_seed)

for prob in max_flow_benchmark:
    problem = prob
num_variables = sum(np.prod(var.shape) for var in problem.variables())
num_constraints = sum(constraint.size for constraint in problem.constraints)
nnz = num_variables # This is the actual number of edges
total_possible_edges = float(n*(n - 1))
density = nnz/total_possible_edges
sparsity = 1 - density
print(f'Generated a valid network with {n} nodes using starting seed {base_seed}')
print(f"Actual Number of Edges: {nnz}")
print(f"Total Possible Edges: {total_possible_edges}")
print(f"Graph sparsity: {sparsity}")
print(f"Graph density: {density}")
print(f"Number of Variables: {num_variables}")
print(f"Number of Constraints: {num_constraints}")

Generated a valid network with 10000 nodes using starting seed 42
Actual Number of Edges: 183993
Total Possible Edges: 99990000.0
Graph sparsity: 0.9981598859885988
Graph density: 0.0018401140114011401
Number of Variables: 183993
Number of Constraints: 193993


In [3]:
# Solve the problem in a standard way using CVXPY
start_time = time.time()
result = problem.solve(solver=cp.OSQP)
end_time = time.time()
solve_time = end_time - start_time
obj_val = problem.value
problem_status = problem.status
num_iters = problem.solver_stats.num_iters
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")

RuntimeError: parallel_for failed: cudaErrorInvalidDevice: invalid device ordinal

In [14]:
P = sp.csc_matrix([[4.0, 1.0], [1.0, 2.0]])
q = np.array([1.0, 1.0])
A = sp.csc_matrix([[1.0, 1.0], [1.0, 0.0], [0.0, 1.0]])
l = np.array([1.0, 0.0, 0.0])
u = np.array([1.0, 0.7, 0.7])

# Make sure we tell OSQP to use the CUDA backend.
solver = osqp.OSQP(algebra="cuda")
solver.setup(P, q, A, l, u, verbose=True, device=3)

# res = solver.solve(device=0)
print("Solver status:", res.info.status)
print("Solution x:", res.x)


RuntimeError: parallel_for failed: cudaErrorInvalidDevice: invalid device ordinal

In [5]:
solver.device

AttributeError: 'OSQP' object has no attribute 'device'